In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torchvision import datasets, transforms
import argparse
import os

In [2]:
#数据加载
from torchvision.datasets import ImageFolder
simple_transform = transforms.Compose([transforms.Resize((256,256))
                                       ,transforms.ToTensor()
                                       ,transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                      ])
train_data = ImageFolder('../chapter4/DogVsCatData/train',simple_transform)
print(train_data.class_to_idx)
print(train_data.classes) 

{'cat': 0, 'dog': 1}
['cat', 'dog']


In [3]:
#定义模型
class DogCat_Net(nn.Module):
    def __init__(self):
        super(DogCat_Net,self).__init__()
        #RGB对应Channel=3，定义16个卷积核，卷积核大小为7，膨胀率为2
        self.conv1 = nn.Conv2d(3, 4, kernel_size=7,dilation=2)
        self.conv2 = nn.Conv2d(4, 16, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(55696, 1000)
        self.fc2 = nn.Linear(1000,50)
        self.fc3 = nn.Linear(50, 2)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 55696)
        x = F.relu(self.fc1(x))
        x = F.dropout(x)
        x = F.relu(self.fc2(x))
        x = F.dropout(x)
        x = self.fc3(x)
        return F.log_softmax(x,dim=1)

In [6]:
def train(args, model, device, dataloader_kwargs):
    global_step = 0
    #手动设置随机种子
    torch.manual_seed(args.get("seed"))
    #加载训练数据
    train_loader = torch.utils.data.DataLoader(args.get("train_data"),batch_size=args.get("batch_size"),num_workers=6,shuffle=True,**dataloader_kwargs)
    
    #使用随机梯度下降进行优化
    optimizer = optim.SGD(model.parameters(), lr=args.get("lr"), momentum=args.get("momentum"))
    #开始训练，训练epoches次
    for epoch in range(1, args.get("epochs") + 1):
        global_step = train_epoch(epoch, args, model, device, train_loader, optimizer,global_step)



def train_epoch(epoch, args, model, device, data_loader, optimizer,global_step):
    #模型转换为训练模式
    model.train()
    pid = os.getpid()
    for batch_idx, (data, target) in enumerate(data_loader):
        #优化器梯度置0
        optimizer.zero_grad()
        #输入特征预测值
        output = model(data.to(device))
        #预测值与标准值计算损失
        loss = F.nll_loss(output, target.to(device))
        #计算梯度
        loss.backward()
        #更新梯度
        optimizer.step()
        #每log_interval步打印一下日志
        if batch_idx % args.get("log_interval") == 0:
            global_step += 1
            viz.line(Y=np.array([loss.item()]), X=np.array([global_step]), update='append', win=args.get("loss_win"))
            print('{}\tTrain Epoch: {} [{}/{} ({:.2f}%)]\tLoss: {:.12f}'.format(pid, epoch, batch_idx * len(data), len(data_loader.dataset),
                                                                               100. * batch_idx / len(data_loader), loss.item()))
    return global_step

In [ ]:
import visdom
import numpy as np
viz = visdom.Visdom(port=8097, server="127.0.0.1",env="Test")
# line updates
loss_win = viz.line(np.arange(1))
#超参数及默认值
args={
    'batch_size':128,
    'epochs':50,
    'lr':0.01,
    'momentum':0.9,
    'seed':1,
    'log_interval':30,
    'train_data':train_data,
    'loss_win':loss_win
}
use_cuda=True if torch.cuda.is_available() else False

if __name__=="__main__":
    #运行时设备
    device = torch.device("cuda" if use_cuda else "cpu")
    torch.cuda.empty_cache()
    #使用固定缓冲区
    dataloader_kwargs = {'pin_memory': True} if use_cuda else {}
    #模型拷贝到设备
    model = DogCat_Net().to(device)
    train(args, model,device, dataloader_kwargs)

### 用Vgg16模型训练

In [3]:
def vgg_train(args, vgg, device, dataloader_kwargs):
    global_step = 0
    #手动设置随机种子
    torch.manual_seed(args.get("seed")+1)
    #加载训练数据
    train_loader = torch.utils.data.DataLoader(args.get("train_data"),batch_size=args.get("batch_size"),num_workers=6,shuffle=True,**dataloader_kwargs)
    
    #使用随机梯度下降进行优化
    optimizer = optim.SGD(vgg.parameters(), lr=args.get("lr"), momentum=args.get("momentum"))
    #开始训练，训练epoches次
    for epoch in range(1, args.get("epochs") + 1):
        global_step = vgg_train_epoch(epoch, args, vgg, device, train_loader, optimizer,global_step)



def vgg_train_epoch(epoch, args, vgg, device, data_loader, optimizer,global_step):
    #模型转换为训练模式
    vgg.train()
    pid = os.getpid()
    for batch_idx, (data, target) in enumerate(data_loader):
        #优化器梯度置0
        optimizer.zero_grad()
        #输入特征预测值
        output = F.log_softmax(vgg(data.to(device)))
        #预测值与标准值计算损失
        loss = F.nll_loss(output.narrow(1,0,2),target.to(device))
        #计算梯度
        loss.backward()
        #更新梯度
        optimizer.step()
        #每10步打印一下日志
        if batch_idx % 10 == 0:
            global_step += 1
            viz.line(Y=np.array([loss.item()]), X=np.array([global_step]), update='append', win=args.get("loss_win"))
            print('{}\tTrain Epoch: {} [{}/{} ({:.2f}%)]\tLoss: {:.12f}'.format(pid, epoch, batch_idx * len(data), len(data_loader.dataset),
                                                                               100. * batch_idx / len(data_loader), loss.item()))
    return global_step

In [4]:
#数据加载
from torchvision.datasets import ImageFolder
from torchvision import datasets,transforms,utils,models
import visdom
import numpy as np
viz = visdom.Visdom(port=8097, server="127.0.0.1",env="Test")
# line updates
loss_win = viz.line(np.arange(1))
simple_transform = transforms.Compose([transforms.Resize((224,224))
                                       ,transforms.ToTensor()
                                       ,transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                      ])
train_data = ImageFolder('../chapter4/DogVsCatData/train',simple_transform)
#用微调好的网络传入数据重新训练模型
#超参数及默认值
args={
    'batch_size':32,
    'epochs':10,
    'lr':0.0001,
    'momentum':0.5,
    'seed':1,
    'log_interval':30,
    'train_data':train_data,
    'loss_win':loss_win
}
use_cuda=True if torch.cuda.is_available() else False

if __name__=="__main__":
    #运行时设备
    device = torch.device("cuda:0" if use_cuda else "cpu")
    vgg16 = models.vgg16(pretrained=True)
    
    #冻结网络层
    for param in vgg16.features.parameters(): 
        param.requires_grad = False                                                              
    #微调网络
    fc_features = vgg16.classifier[6].in_features  
    #修改类别为2
    vgg16.classifier[6] = nn.Linear(fc_features,2) 
    vgg16.cuda()
    #使用固定缓冲区
    dataloader_kwargs = {'pin_memory': True} if use_cuda else {}
    vgg_train(args,vgg16,device, dataloader_kwargs)

D:\softwares\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


20632	Train Epoch: 1 [0/22587 (0.00%)]	Loss: 0.671515882015
20632	Train Epoch: 1 [320/22587 (1.42%)]	Loss: 0.599117100239
20632	Train Epoch: 1 [640/22587 (2.83%)]	Loss: 0.579360485077
20632	Train Epoch: 1 [960/22587 (4.25%)]	Loss: 0.489593654871
20632	Train Epoch: 1 [1280/22587 (5.67%)]	Loss: 0.371161013842
20632	Train Epoch: 1 [1600/22587 (7.08%)]	Loss: 0.342235237360
20632	Train Epoch: 1 [1920/22587 (8.50%)]	Loss: 0.318746000528
20632	Train Epoch: 1 [2240/22587 (9.92%)]	Loss: 0.283182859421
20632	Train Epoch: 1 [2560/22587 (11.33%)]	Loss: 0.286416947842
20632	Train Epoch: 1 [2880/22587 (12.75%)]	Loss: 0.275277584791
20632	Train Epoch: 1 [3200/22587 (14.16%)]	Loss: 0.283171862364
20632	Train Epoch: 1 [3520/22587 (15.58%)]	Loss: 0.288028866053
20632	Train Epoch: 1 [3840/22587 (17.00%)]	Loss: 0.295131027699
20632	Train Epoch: 1 [4160/22587 (18.41%)]	Loss: 0.174489945173
20632	Train Epoch: 1 [4480/22587 (19.83%)]	Loss: 0.187735587358
20632	Train Epoch: 1 [4800/22587 (21.25%)]	Loss: 0.195

20632	Train Epoch: 2 [17920/22587 (79.32%)]	Loss: 0.085207156837
20632	Train Epoch: 2 [18240/22587 (80.74%)]	Loss: 0.062715381384
20632	Train Epoch: 2 [18560/22587 (82.15%)]	Loss: 0.196202084422
20632	Train Epoch: 2 [18880/22587 (83.57%)]	Loss: 0.134793311357
20632	Train Epoch: 2 [19200/22587 (84.99%)]	Loss: 0.044903233647
20632	Train Epoch: 2 [19520/22587 (86.40%)]	Loss: 0.037119306624
20632	Train Epoch: 2 [19840/22587 (87.82%)]	Loss: 0.044805184007
20632	Train Epoch: 2 [20160/22587 (89.24%)]	Loss: 0.067375332117
20632	Train Epoch: 2 [20480/22587 (90.65%)]	Loss: 0.028314827010
20632	Train Epoch: 2 [20800/22587 (92.07%)]	Loss: 0.097856707871
20632	Train Epoch: 2 [21120/22587 (93.48%)]	Loss: 0.028983391821
20632	Train Epoch: 2 [21440/22587 (94.90%)]	Loss: 0.036553006619
20632	Train Epoch: 2 [21760/22587 (96.32%)]	Loss: 0.032814148813
20632	Train Epoch: 2 [22080/22587 (97.73%)]	Loss: 0.017245352268
20632	Train Epoch: 2 [22400/22587 (99.15%)]	Loss: 0.116387799382
20632	Train Epoch: 3 [0/2

KeyboardInterrupt: 